**6. (25%, coding assignment) Could you implement your own KNN function from scratch (without calling any existing classifier packages)? Your own KNN function must have following parameters to tune: n_neighbors, weights. Can you compare your own classifier with sklearn neighbors.KNeighborsClassifier in terms performance, complexity, etc. Data usage: datasets.load_iris() from sklearn.**

In [ ]:
import math
import numpy as np
import pandas as pd

class CustomKNN:

  def __init__(self, n_neighbors=5, weight='uniform', power=2, distance='minkowski'):
    
    if weight not in ['uniform', 'distance']:
      raise IOError('Unrecognized weight type')
    if not isinstance(power, int):
      raise IOError('power should be an integer')
    if distance not in ['minkowski', 'euclidean', 'manhattan', 'cosine']:
      raise IOError('Distance is not recognized')

    self._n_neighbors = n_neighbors
    self._weight = weight
    self._distance = distance
    if distance == 'manhattan':
      self._power = 1
    elif distance == 'euclidean':
      self._power = 2
    else:
      self._power = power

  # Showing the parameters used in this classifier
  def get_params(self):
    params = {
        'n_neighbors': self._n_neighbors,
        'weight': self._weight,
        'power': self._power,
        'distance': self._distance
    }
    return params

  def euclidean_distance(self, query, data):
    ed = sum([(query[i] - data[i]) ** 2 for i in range(len(query))])
    ed = math.sqrt(ed)
    return ed

  def manhattan_distance(self, query, data):
    md = sum([math.abs(query[i] - data[i]) for i in range(len(query))])
    return md

  def minkowski_distance(self, query, data):
    if self._power == 1:
      return self.manhattan_distance(query, data)
    elif self._power == 2:
      return self.euclidean_distance(query, data)
    md = [math.pow(query[i] - data[i], self._power) for i in range(len(query))]
    md = sum(md)
    md = math.pow(md, 1 / self._power)
    return md

  def cosine_similarity(self, query, data):
    cs = sum([query[i] * data[i] for i in range(len(query))])
    deno_q = math.sqrt(sum([q ** 2 for q in query]))
    deno_d = math.sqrt(sum([d ** 2 for d in data]))
    cs = cs / (deno_q * deno_d)
    return cs

  def find_neighbors(self, query, dataset):
    distances = []
    distance_function = self.cosine_similarity if self._distance == 'cosine' else self.minkowski_distance
    for data in dataset:
      dist = distance_function(query, data)
      distances.append((data, dist))
    distances.sort(key=lambda t: t[1])

    neighbors = []
    for i in range(self._n_neighbors):
      neighbors.append(distances[i][0])
    return neighbors

  # Implemented to find the indices from the descriptive features 
  # to find the targets
  def compare_sublist(self, des_fea, nei_fea):
    nei_idx = []
    for des in des_fea:
      # all neighbors are removed from the list, stop searching
      if nei_fea is None:
        break

      rmb = None
      for nei in nei_fea:
        if all([des[i] == nei[i] for i in range(len(nei))]):
          # Remember the neighbor index that is found
          rmb = nei_fea.index(nei)
          nei_idx.append(des_fea.index(des))
          break
      # Pop the searched neighbor to reduce iterations
      if rmb:
        nei_fea.pop(rmb)

    return nei_idx

  def flatten(self, lst):
    res = []
    for it in lst:
        if isinstance(it, list):
            res.extend(self.flatten(it))
        else:
            res.append(it)
    return res

  # Simple check to make sure all values in the descriptive features are numerical
  def validate_inputs(self, des_fea, queries):
    flattened = self.flatten(des_fea)
    return not any([isinstance(v, str) for v in flattened])

  # Improved ChatGPT suggestion
  # Prompt: can you suggest a way to convert any iterable that is not a list type into a list in Python?
  def to_list(self, it):
    if isinstance(it, list):
        return it
    elif isinstance(it, tuple) or isinstance(it, set):
        return list(it)
    elif isinstance(it, dict):
        return list(it.values())
    elif isinstance(it, pd.DataFrame):
        return it.values.tolist()
    elif isinstance(it, np.ndarray):
        return it.tolist()
    else:
        raise TypeError("Object is not iterable")

  def predict(self, queries, descriptives, targets):
    queries = self.to_list(queries)
    descriptives = self.to_list(descriptives)
    if not self.validate_inputs(descriptives, queries):
      raise IOError('Descriptive features must be numerical')

    targets = self.to_list(targets)
    target_type = 'c' if len(np.unique(targets)) <= 5 else 'r'  # check what kind of prediction to make

    predictions = []
    for query in queries:
      neighbors = self.find_neighbors(query, descriptives)
      neighbors_idx = self.compare_sublist(descriptives, neighbors)

      outputs = [targets[idx] for idx in neighbors_idx]
      if target_type == 'c':
        predictions.append(max(set(outputs), key=outputs.count))
      else:
        predictions.append(sum(outputs) / len(outputs))
    return predictions

In [ ]:
from sklearn import neighbors, datasets

iris = datasets.load_iris()

X = iris.data[:, :2]
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

# print(X_train)

The following sections will be comparing the complexity and performance of the custom KNN classifier and the KNN classifier from Scikit learn

In [ ]:
import time
from sklearn.metrics import accuracy_score
cknn = CustomKNN()

print("Parameters")
print(cknn.get_params())

start_time = time.time()
y_pred = cknn.predict(X_test, X_train, y_train)
end_time = time.time()
print("Total execution time: {}".format(end_time - start_time))
print("Accuracy score: {}".format(accuracy_score(y_pred, y_test)))

Parameters
{'n_neighbors': 5, 'weight': 'uniform', 'power': 2, 'distance': 'minkowski'}
Total execution time: 0.032500267028808594
Accuracy score: 0.7631578947368421


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

print("Parameters")
print(knn.get_params())

knn.fit(X_train, y_train)

start_time = time.time()
y_pred = knn.predict(X_test)
end_time = time.time()
print("Total execution time: {}".format(end_time - start_time))
print("Accuracy score: {}".format(accuracy_score(y_pred, y_test)))



print("\nBrute force")
knn_br = KNeighborsClassifier(algorithm='brute')

print("Parameters")
print(knn.get_params())

knn_br.fit(X_train, y_train)

start_time = time.time()
y_pred = knn_br.predict(X_test)
end_time = time.time()
print("Total execution time: {}".format(end_time - start_time))
print("Accuracy score: {}".format(accuracy_score(y_pred, y_test)))


Parameters
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Total execution time: 0.0033164024353027344
Accuracy score: 0.7631578947368421

Brute force
Parameters
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
Total execution time: 0.09224963188171387
Accuracy score: 0.7631578947368421


For the accuracy performance, under the same parameters, the custom KNN classifer performs as well as the scikit learn KNN classifier. Since the custom KNN modifier uses a brute force approach to find the neighbors, the accuracy of the custom classifier should, in most cases, perform the same as the scikit learn model.

For the custom KNN classifier, it is using brute force to find the neighbors. The first scikit learn KNN classifier uses a automatic algorithm selector to find neighbors. The execution time on the prediction is 10 times faster than the custom KNN classifers. However, for the second KNN classifier who I specified the algorithm to be 'brute', the execution time is almost 3 times slower than the custom KNN classifier. The complexity for the custom KNN classifier is O(n * k * d * q), where n is the number of observations, d is the number of features, q is the number of queries and k is the number of neighbors. This time is mostly spent on finding the neighbors per query, and when locating the indicies for finding the targets. According to the documentation, the complexity of scikit learn KNN classifier is O(n * d). This is the complexity spent on training. In comparison, the scikit learn KNN model will outpeform the custom KNN model in most cases. I think the algorithm it chooses is a big factor of the time complexity.

For the memory complexity, the custom KNN classifier does not store any datasets. Therefore, the memory complexity is O(1). For the scikit learn KNN classifier, since it stores the dataset into the classifier, the complexity is O(n * d).

